In [1]:
import numpy as np
from pandas import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from tokenize_uk.tokenize_uk import tokenize_words
import pymorphy2

In [2]:
df = pd.read_csv('data/input/comments.tsv', sep='\t', index_col=0)
df_tones = pd.read_csv('data/input/tone-dict-uk.tsv', sep='\t', index_col=0, names=['tone'])
df = df.fillna('')
print('rows count:', len(df))
df.head()

rows count: 14949


,rating,item_bought,review,pros,cons,upvotes,downvotes
id,,,,,,,
23415273,5,1,"сервіс відмінний, морозильна камера працює добре.",,,0,0
33284346,5,1,гарна та якісна морозильна камера. при першому...,недорогий,нема,0,0
33040878,4,1,"камеру привезли, все працює, все добре, все як...",,немає,2,0
38902893,5,1,справляється із своєю роботою,ціна якість,,0,0
20357268,5,1,"користуюсь міс., дуже задоволений!","все чудово працює, тихий.",поки не виявлено.,0,0


In [3]:
stopwords = set(["а","або","але","б","без","би","бо","був","буде","була","були","було","бути","в","вам","вами","вас","ваш","ваша","ваше","вашим","вашими","ваших","ваші","вашій","вашого","вашої","вашому","вашою","вашу","вже","ви","від","він","вона","вони","воно","всі","де","для","до","дуже","є","з","за","зі","і","із","її","їй","їм","їх","й","його","йому","ким","кого","коли","кому","лише","має","мене","мені","ми","мій","мною","мого","моє","моєї","моєму","моєю","можна","мої","моїй","моїм","моїми","моїх","мою","моя","на","нам","нами","нас","наш","наша","наше","нашим","нашими","наших","наші","нашій","нашого","нашої","нашому","нашою","нашу","неї","нею","ним","ними","них","ній","нім","ну","нього","ньому","під","після","по","при","про","саме","себе","собі","та","так","також","там","твій","твого","твоє","твоєї","твоєму","твоєю","твої","твоїй","твоїм","твоїми","твоїх","твою","твоя","те","тебе","ти","тим","тими","тих","ті","тієї","тією","тій","тільки","тім","то","тобі","тобою","того","тоді","той","тому","ту","тут","у","хто","це","цей","ці","цього","цьому","через","чи","чиє","чиєї","чиєму","чиї","чиїй","чиїм","чиїми","чиїх","чий","чийого","чийому","чим","чию","чия","чого","чому","що","щоб","щодо","щось","я","як","яка","який","які","якщо"])

In [4]:
morph = pymorphy2.MorphAnalyzer(lang='uk')
def tokenizer(string):
    return [morph.parse(word)[0].normal_form for word in tokenize_words(string) if word and word not in stopwords]

def sentiment_tokenizer(tokens):
    result = []
    for token in tokens:
        if token:
            if token in df_tones.index:
                result.append(df_tones.at[token, 'tone'])
            else:
                result.append(token)
    return result

In [5]:
text_clf = Pipeline([
    ('vect', CountVectorizer(analyzer='word')),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

svm_clf = Pipeline([
    ('vect', CountVectorizer(analyzer='word')),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss = 'hinge', penalty = 'l2', 
                          alpha = 1e-4, random_state = 1,
                          max_iter = 50, tol = None)),
])

### Using tones

In [6]:
df_with_tones = df.copy()
df_with_tones['review_tones'] = df_with_tones['review'].apply(tokenizer).apply(sentiment_tokenizer).str.join('_rev ')
df_with_tones = df_with_tones.dropna(subset=['review_tones'])
x_train_tones, x_test_tones, y_train_tones, y_test_tones = train_test_split(df_with_tones['review_tones'], df_with_tones['rating'], test_size=0.3, random_state=1)

text_clf.fit(x_train_tones, y_train_tones)
predicted = text_clf.predict(x_test_tones)
np.mean(predicted == y_test_tones)

0.6224662162162162

In [14]:
svm_clf.fit(x_train_tones, y_train_tones)
predicted = svm_clf.predict(x_test_tones)
np.mean(predicted == y_test_tones)

0.5962837837837838

### Without tones

In [8]:
reviews = df['review'].apply(tokenizer).str.join('_rev ')
x_train, x_test, y_train, y_test = train_test_split(reviews, df['rating'], test_size=0.3, random_state=1)

In [9]:
text_clf.fit(x_train, y_train)
predicted = text_clf.predict(x_test)
np.mean(predicted == y_test)

0.6831661092530658

In [15]:
svm_clf.fit(x_train, y_train)
predicted = svm_clf.predict(x_test)
np.mean(predicted == y_test)

0.7076923076923077

### adding pros and cons

In [11]:
pros = df['pros'].apply(tokenizer).str.join('_pros ')
cons = df['cons'].apply(tokenizer).str.join('_cons ')
with_adv = reviews.str.cat([pros, cons], sep=' ')

x_train_adv, x_test_adv, y_train_adv, y_test_adv = train_test_split(with_adv, df['rating'], test_size=0.3, random_state=1)

svm_clf.fit(x_train_adv, y_train_adv)
predicted_adv = svm_clf.predict(x_test_adv)
np.mean(predicted_adv == y_test_adv)

0.725752508361204

### adding votes

In [12]:
upvotes = df['upvotes'].astype(str) + '_upv'
downvotes = df['downvotes'].astype(str) + '_down'

with_votes = with_adv.str.cat([upvotes, downvotes], ' ')

x_train_votes, x_test_votes, y_train_votes, y_test_votes = train_test_split(with_votes, df['rating'], test_size=0.3, random_state=1)

svm_clf.fit(x_train_votes, y_train_votes)
predicted_votes = svm_clf.predict(x_test_votes)
np.mean(predicted_votes == y_test_votes)

0.7261984392419175

### considering whether the item was bought

In [13]:
item_bought = df['item_bought'].astype(str) + '_bought'
with_bought = with_adv.str.cat(item_bought, sep=' ')

x_train_bought, x_test_bought, y_train_bought, y_test_bought = train_test_split(with_bought, df['rating'], test_size=0.3, random_state=1)

svm_clf.fit(x_train_bought, y_train_bought)
predicted_bought = svm_clf.predict(x_test_bought)
np.mean(predicted_bought == y_test_bought)

0.7264214046822742